In [ ]:
import numpy as np
import pandas as pd


def balanced_map(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Python port of balanced_map(df) from `balanced_map.R`.

    Parameters
    ----------
    df : pd.DataFrame
        Must contain columns: ID_XX, T_XX, tsfilled_XX.

    Returns
    -------
    out : pd.DataFrame
        Wide DataFrame: columns are ['ID_XX', 'H_<t1>', 'H_<t2>', ...].
    '''
    required = {'ID_XX', 'T_XX', 'tsfilled_XX'}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"balanced_map: missing columns {sorted(missing)}")

    d = df.loc[:, ['ID_XX', 'T_XX', 'tsfilled_XX']].copy()
    # R: H_t_XX <- as.numeric(1 - tsfilled_XX)
    d['H_t_XX'] = (1.0 - pd.to_numeric(d['tsfilled_XX'], errors='coerce')).astype(float)
    d = d.drop(columns=['tsfilled_XX'])

    d = d.sort_values(['ID_XX', 'T_XX'], kind='mergesort')
    d['H_t_m_1_XX'] = d.groupby('ID_XX')['H_t_XX'].shift(1)

    # R: subset(df, !is.na(H_t_m_1_XX))
    d = d.loc[~d['H_t_m_1_XX'].isna()].copy()

    # R: H_t <- as.numeric(H_t_XX==1 & H_t_m_1_XX==1)
    d['H_t'] = ((d['H_t_XX'] == 1) & (d['H_t_m_1_XX'] == 1)).astype(float)
    d = d.drop(columns=['H_t_XX', 'H_t_m_1_XX'])

    # Preserve time ordering like R's factor levels
    times = pd.Index(pd.unique(df['T_XX']))
    wide = d.pivot(index='ID_XX', columns='T_XX', values='H_t')
    wide = wide.reindex(columns=times, fill_value=np.nan)

    wide.columns = [f"H_{t}" for t in wide.columns]
    wide = wide.reset_index(drop=False)
    wide.index = range(1, len(wide) + 1)  # R uses 1:nrow
    return wide


# --- small demo ---
if __name__ == '__main__':
    demo = pd.DataFrame({
        'ID_XX': [1,1,1,2,2,2],
        'T_XX':  [1,2,3,1,2,3],
        'tsfilled_XX': [1,0,0, 1,1,0],
    })
    print(balanced_map(demo))
